# MasterMonitor Sensor Script

## AdaFruit IO

### Credentials

In [1]:
from Adafruit_IO import Client, Feed, RequestError
from dotenv import load_dotenv
import os

load_dotenv()

AdaFruitIO_Username = os.environ.get("ADAFRUIT_IO_USERNAME")
AdaFruitIO_Key = os.environ.get("ADAFRUIT_IO_KEY")

aio = Client(AdaFruitIO_Username, AdaFruitIO_Key)

### Feeds

In [2]:
def getFeed(name):
    try:
        feedID = aio.feeds(name)
    except RequestError:
        feed = Feed(name=name)
        feedID = aio.create_feed(feed)
    return feedID

soundFeed = getFeed("sound")
luxFeed = getFeed("light")
intTempFfeed = getFeed("inttempf")
extemp2Feed = getFeed("extemp2")
humidFeed = getFeed("humidity")
pressFeed = getFeed("pressure")
tvocsFeed = getFeed("new-sensor.tvocs")
co2Feed = getFeed("new-sensor.co2")
msgFeed = getFeed("new-sensor.messages")

## Google Sheets

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

sheet_name = "enviro-pi_log"

scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scopes)
client = gspread.authorize(creds)

# Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).worksheet("Values")

## BME280 Sensor

In [4]:
from bme280 import BME280

bme280 = BME280()

## External Temperature Sensor

In [5]:
import smbus

def readTemp():
    address = 0x48
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

## CCS811 Sensor

In [6]:
import busio
from board import I2C
from adafruit_ccs811 import CCS811

i2c = I2C() 
ccs = CCS811(i2c)

while (not ccs.data_ready):
    pass

print ("CCS811 Sensor ready.")

CCS811 Sensor ready.


## Light Sensor

In [7]:
from ltr559 import LTR559

ltr559 = LTR559()

## Noise Sensor

In [8]:
from enviroplus.noise import Noise

noise = Noise()

## Main

### Imports

In [9]:
import ST7735

import time
from IPython.display import clear_output
from board import *
from datetime import datetime

### Variables

In [10]:
try:
    metadata = {'lat': 36.017433,
           'lon': -84.239815,
           'ele': 0,
           'created_at': None}
    
    update = 300
    update_data = 60.0
    update_msg  = 1800.0
    update_lcd  = 3.0
    next_lcd    = 't'
    count = 0
    
    running = True
    
    tvoc_alert = False
    tvoc_alert_level = 1500
    
    dashboard_url = "https://io.adafruit.com/Thuviksa/dashboards/weather-monitor"

except IOError:
    print("Just an IO Error, nothing to worry 😉😉😉")
    pass

except RuntimeError:
    print("Not Good 👎👎👎")
    pass

### Loop

In [12]:
start_time   = time.time()
current_time = start_time

count = 0

while running:
    try:
        if ((int (current_time - start_time) % update_data) == 0):
            interc = bme280.get_temperature()
            interf = interc * 1.8 + 32
            humid = bme280.get_humidity()
            press = bme280.get_pressure()
            co2 = CCS811(i2c).eco2
            tvocs = CCS811(i2c).tvoc
            lux = ltr559.get_lux()
            prox = ltr559.get_proximity()
            range=[(100, 1200)]
            amps = noise.get_amplitudes_at_frequency_ranges(range)
            extc = readTemp()
            extf = extc * 1.8 + 32
            
            print(tvocs)
            
            clear_output(wait=True)

            ltr559.update_sensor()
            output = "External-Temp: {0:.2f} C - {1:.2f} F".format(extc, extf)
            output+= "Internal-Temp: {0:.2f} C, {1:.2f} F".format(interc, interf)
            output+= "Humidity: {0:.2f} % - Pressure: {1:.2f} hPa".format(humid, press)
            output+= "Light: {0:.2f} - Noise: {1:.2f} - Proximity: {2:.2f}".format(lux, amps[0], prox)
            #output+= "CO2: {0:.2f} ppm - TVOCs: {1:.2f} ppb".format(co2, tvocs)
            print(output)
            print('Updated: ', datetime.now())
            
            aio.send_data(co2Feed.key, co2, metadata)
            aio.send_data(tvocsFeed.key, tvocs, metadata)
            aio.send_data(soundFeed.key, amps[0], metadata)
            aio.send_data(luxFeed.key, lux, metadata)
            aio.send_data(intTempCfeed.key, inter.c, metadata)
            aio.send_data(intTempFfeed.key, inter.f, metadata)
            aio.send_data(extemp2Feed.key, ext.f, metadata)
            aio.send_data(extempFeed.key, ext.c, metadata)
            aio.send_data(humidFeed.key, humid, metadata)
            aio.send_data(pressFeed.key, pressure, metadata)
            
            now = datetime.now()
            nowtime = now.strftime("%H:%M:%S")
            nowday = now.strftime("%m/%d/%Y")
            row = [nowday, nowtime, inter.c, inter.f, ext.c, ext.f, humid, press, lux, amps[0], co2, tvocs]
            sheet.append_row(row)
            
            if(v > tvoc_alert_level):
                if(not tvoc_alert):
                    tvoc_alert=True
                    msg="TVOC Reading is over *{0:.2f}* ppb ⚠⚠⚠".format(tvoc_alert_level)
                    aio.send_data(msgFeed.key, msg, metadata)
            else:
                if(tvoc_alert):
                    tvoc_alert=False
                    msg="TVOC Reading returned to below *{0:.2f}* ppb 😀😃😃 ".format(tvoc_alert_level)
                    aio.send_data(msgFeed.key, msg, metadata)
        
        count = count + 1
        
        if (count > 5):
            print("Update message")
            msg = "_Thuviksa's Updates_"
            msg+="<br>Internal-Temp: *{0:.2f} C* - *{1:.2f} F*".format(interc,interf)
            msg+="<br>External-Temp: *{0:.2f} C* - *{01:.2f} F*".format(extc, extf)
            msg+="<br>Humidity: *{0:.2f} %* - Pressure: *{1:.2f} hPa*".format(humid,press)
            msg+="<br>Air quality: CO2: *{0:.2f} ppm* - TVOC: *{1:.2f} ppb*<br>".format(co2,tvocs)
            msg+="<br> "
            msg+="<br>Full dashboard: {}".format(dashboard_url)
            aio.send_data(msgFeed.key, msg, metadata)
        
        time.sleep(update)
        current_time = time.time()


            
        print("Sending Data to the AdaFruit Cloud......")

    except IOError:
        print("Just an IO Error, nothing to worry 😉😉😉")
        pass
    
    except KeyboardInterrupt:
        print("Loop Terminated by User 👾👾👾")
        break
        
    except RuntimeError:
        print("Not Good  👎👎👎")
        pass

External-Temp: 22.25 C - 72.05 FInternal-Temp: 29.39 C, 84.90 FHumidity: 53.97 % - Pressure: 985.46 hPaLight: 12.43 - Noise: 0.20 - Proximity: 0.00
Updated:  2021-10-23 18:26:48.680677


RequestError: Adafruit IO request failed: 422 Unprocessable Entity - request failed - Record invalid. Failed to save data to Thuviksa/feeds/new-sensor.tvocs, data missing required value